In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to your data directories
cracked_dir = '/kaggle/input/decksbridge/Decks/Cracked'
non_cracked_dir = '/kaggle/input/decksbridge/Decks/Non-cracked'

# Function to get image file paths
def get_image_paths(folder, label):
    image_paths = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith('.jpg'):
            image_paths.append(os.path.join(folder, filename))
            labels.append(label)
    return image_paths, labels

cracked_paths, cracked_labels = get_image_paths(cracked_dir, 1)  # 1 for cracked
non_cracked_paths, non_cracked_labels = get_image_paths(non_cracked_dir, 0)  # 0 for non-cracked

# Combine and split the data
image_paths = np.array(cracked_paths + non_cracked_paths)
labels = np.array(cracked_labels + non_cracked_labels)

# Split data into training and testing sets
X_train_paths, X_test_paths, y_train, y_test = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Data generator for loading images in batches
def image_data_generator(image_paths, labels, batch_size, is_training=True):
    while True:
        idx = np.random.permutation(len(image_paths))
        image_paths = image_paths[idx]
        labels = labels[idx]
        
        for start in range(0, len(image_paths), batch_size):
            end = min(start + batch_size, len(image_paths))
            batch_paths = image_paths[start:end]
            batch_labels = labels[start:end]
            
            images = []
            for path in batch_paths:
                img = Image.open(path)
                img = img.resize((224, 224))  # Resize images to 224x224
                img = np.array(img) / 255.0  # Normalize images
                images.append(img)
            
            yield np.array(images), np.array(batch_labels)

# Create data generators
batch_size = 1024 # You can adjust this based on your available memory
train_generator = image_data_generator(X_train_paths, y_train, batch_size)
validation_generator = image_data_generator(X_test_paths, y_test, batch_size, is_training=False)

2024-07-11 11:06:56.769351: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 11:06:56.769504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 11:06:56.939269: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define the model architecture
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)  # Regression output

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model using the generators with early stopping
steps_per_epoch = len(X_train_paths) // batch_size
validation_steps = len(X_test_paths) // batch_size

model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=5,  # Set a high number of epochs to allow for early stopping
    callbacks=[early_stopping]
)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 673s 68s/step - loss: 0.2319 - mae: 0.3191 - val_loss: 0.1295 - val_mae: 0.2958
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 596s 63s/step - loss: 0.1321 - mae: 0.2674 - val_loss: 0.1346 - val_mae: 0.2328
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 600s 61s/step - loss: 0.1273 - mae: 0.2528 - val_loss: 0.1338 - val_mae: 0.2201
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 607s 62s/step - loss: 0.1299 - mae: 0.2397 - val_loss: 0.1309 - val_mae: 0.2408
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 598s 61s/step - loss: 0.1236 - mae: 0.2492 - val_loss: 0.1229 - val_mae: 0.2182


In [3]:
def predict_remaining_life(image_path):
    # Load and preprocess the image
    img = Image.open(image_path)
    img = img.resize((224, 224))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Predict the remaining life
    remaining_life = model.predict(img)
    return remaining_life[0][0]

# Example usage
image_path = '/kaggle/input/decksbridge/Decks/Cracked/7001-17.jpg'
predicted_life = predict_remaining_life(image_path)
print(f'Predicted remaining life of the bridge: {predicted_life} years')
image_path = '/kaggle/input/decksbridge/Decks/Non-cracked/7001-107.jpg'
predicted_life = predict_remaining_life(image_path)
print(f'Predicted remaining life of the bridge: {predicted_life} years')

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted remaining life of the bridge: 0.10663887858390808 years
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Predicted remaining life of the bridge: 0.10664737969636917 years
